# Journal lists for the field of german law

# Table of content
* [Configure Notebook](#configure)
* [Intro](#Intro)
    * [General](#General)
    * [Specifications](#Specifications)
* [Journal list from different data sources](#Journal-list-from-different-data-sources)
    * [WoS](#list-wos)
    * [Scopus](#list-scopus)
    * [OpenAlex](#list-openalex)
* [Coverage of the gold standard list in different data sources](#coverage)
    * [Preparations](#coverage-preparations)
    * [Coverage](#coverage-head)
        * [Web of Science](#coverage-wos)
        * [Scopus](#coverage-scp)
        * [OpenAlex](#coverage-openalex)
        * [CrossRef](#coverage-crossref)
    * [Coverage results](#coverage-results)
    

Install and load JupySQL (https://jupysql.ploomber.io)

# <a id="configure"></a>Configure Notebook

In [2]:
%pip install jupysql python-dotenv --quiet
%load_ext sql
%config SqlMagic.displaylimit = 20
%load_ext autoreload
%pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Configure the sql adapter with your credentials. This requires an `.env` file having the following entries:
```
KB_HOST='<host name as as per KB instructions>'
KB_DB='<database name as per KB instructions>'
KB_PORT=<port as per KB instructions>
KB_USER='<Your user name>'
KB_PASS='<Your password>'
```

You also need to connect to the KB via VPN as per KB instructions.

In [3]:
from dotenv import load_dotenv
from sqlalchemy import URL, create_engine
import os
load_dotenv('.env')
url = URL.create(
    "postgresql",
    host=os.environ['KB_HOST'],
    port=os.environ['KB_PORT'],
    database=os.environ['KB_DB'],
    username=os.environ['KB_USER'],
    password=os.environ['KB_PASS']
)
engine = create_engine(url)
%sql engine --alias kb_prod

Now you can run sql queries directly in the python notebook.

# <a id="Intro"></a>Intro

## <a id="General"></a>General

This notebook is for creating lists of journals that are considered as 'german law' journals in different sources. 
Therefore we looked at attributes in the different sources that define
* language = 'german'
* field = 'law'
The exact attributes and values concerning this content vary among the data sources. We used the sources
* Web of Science (wos)
* Scopus (scp)
* OpenAlex (openalex)

In a second step we used a given list (from an article Gröls/Gröls) with journals considered as relevant for the field of german law in the eyes of experts. We checked coverage of these journals in the data sources.

## <a id="Specifications"></a>Specifications

To run this, you have to specify the version you want to use from the different sources. 
The version parameter specifies a suffix that is appended to all result tables. For Web of Science (wos) and Scopus (scp) the KB (https://bibliometrie.info/) processes quarterly bibliometric databases in a specific schema. For OpenAlex, data is processed monthly. 

The parameters wos/scopus/openalex_bdb_schema have to be names of schemas in the PostgreSQL database following the KB data schema for bibliometric databases (see https://zenodo.org/records/13935407). The queries create tables in the given result schema. 

In [4]:
version_name = "202408"
result_schema = "project_rewi"
wos_bdb_schema = "wos_b_202407"
scp_bdb_schema = "scp_b_202407"
openalex_bdb_schema = "fiz_openalex_bdb_20240831_openbib"
openalex_rep_schema = "fiz_openalex_rep_20240831_openbib"
crossref_table = "open_add_ons.cr_oct2023"

# <a id="Journal-list-from-different-data-sources"></a> Journal list from different data sources

General Specifications: 
* all journals with at least one article with: class 'Law', source type journal, german language
* from this result only the journals with at least 10 german law articles and at least 10% of all items german law article

This specifications are set based on values given in the respective database.

## <a id="list-wos"></a> WoS

Specifications: 
* all journals with at least one item with: class_name = 'Law', source type = 'Journal', item type = 'Article', language = 'ger' --> {{result_schema}}.eva_wos_de_law_journals_trivial_search_{{version_name}}
* from this result only the journals with at least 10 german law articles and at least 10% of all items german law article

Check for the specifications (which are the appropiate values available for class_name):

In [4]:
%%sql
select * 
from {{wos_bdb_schema}}.classification_system 
where lower(class_name) like '%law%' or lower(class_name) like '%just%'

Running query in 'kb_prod'

1 rows affected.

class_system,scopus_class_code,class_name
Subject Categories,None,Law


Get all the 'law items': 

In [5]:
%%sql
drop table if exists {{result_schema}}.tmp_wos_law_items_{{version_name}};
create table {{result_schema}}.tmp_wos_law_items_{{version_name}} as (
select item_id, doi, pubyear, lower(source_title) as source_title, publisher_hash, languages 
from {{wos_bdb_schema}}.items 
where 'Law' =any(class_name) and source_type='Journal' and 'Article' = any(item_type)
);

Running query in 'kb_prod'

158418 rows affected.

++
||
++
++

Which languages do the law items have? 

In [6]:
%%sql
select languages, count(distinct item_id) as item_count, count(distinct source_title) as source_title_count 
from {{result_schema}}.tmp_wos_law_items_{{version_name}}  
group by languages;

Running query in 'kb_prod'

18 rows affected.

languages,item_count,source_title_count
[],2,2
['afr'],10,1
['dut'],26,2
['eng'],156857,250
"['eng', 'est']",1,1
['fre'],109,11
['ger'],118,2
['hrv'],1,1
['ita'],6,2
['lat'],1,1


In [7]:
%%sql
drop table if exists {{result_schema}}.tmp_wos_law_de_items_{{version_name}};
create table {{result_schema}}.tmp_wos_law_de_items_{{version_name}} as (
select item_id, doi, pubyear, lower(source_title) as source_title, publisher_hash 
from {{wos_bdb_schema}}.items  
where 'ger' = any(languages) and 'Law' =any(class_name) and source_type='Journal' and 'Article' = any(item_type)
);
create index on {{result_schema}}.tmp_wos_law_de_items_{{version_name}}(item_id);
create index on {{result_schema}}.tmp_wos_law_de_items_{{version_name}}(publisher_hash);

drop table if exists {{result_schema}}.tmp_wos_law_de_journals_{{version_name}};
create table {{result_schema}}.tmp_wos_law_de_journals_{{version_name}} as (
select lower(b.source_title) as source_title, array_agg(distinct sn_c) as issn, 
count(distinct b.item_id) as article_count, count(distinct a.item_id) as law_and_de, 
sum(cit_all_years) as cited_by_count, min(b.pubyear) as start_pubyear, max(b.pubyear) as end_pubyear
from {{result_schema}}.tmp_wos_law_de_items_{{version_name}} a 
join {{wos_bdb_schema}}.items b 
on a.source_title = lower(b.source_title) and 'Article' = any(b.item_type) -- alle items deren source title in den law und de items auftaucht
left join {{wos_bdb_schema}}.issn_isbn c 
on c.item_id = b.item_id 
group by lower(b.source_title)
);

drop table if exists {{result_schema}}.eva_wos_de_law_journals_trivial_search_{{version_name}};
create table {{result_schema}}.eva_wos_de_law_journals_trivial_search_{{version_name}} as (
select source_title, issn, 
article_count, cited_by_count, law_and_de, 
case when article_count > 0 then round(law_and_de/article_count::decimal,2) else null end as law_and_de_article_share, 
case when article_count > 0 then round(cited_by_count/article_count::decimal,2) else null end as citations_per_paper,
    start_pubyear, end_pubyear
from  {{result_schema}}.tmp_wos_law_de_journals_{{version_name}});

Running query in 'kb_prod'

118 rows affected.

2 rows affected.

2 rows affected.

++
||
++
++

This is the intermediate result with the share of German law articles that has to be filtered again according to the specifications 
(at least 10 german law artices, at least 10% german law articles among all articles from the journal):

In [8]:
%%sql
drop table if exists {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}};
create table {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}} as (
select * 
from {{result_schema}}.eva_wos_de_law_journals_trivial_search_{{version_name}}  
where law_and_de >= 10 and law_and_de_article_share >= 0.1 
order by citations_per_paper desc
);

Running query in 'kb_prod'

2 rows affected.

++
||
++
++

Get the number of resulting journals:

In [9]:
%%sql
select count(*) as number_of_journals from  {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

1 rows affected.

number_of_journals
2


Get the details: 

In [10]:
%%sql
select * from  {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

2 rows affected.

source_title,issn,article_count,cited_by_count,law_and_de,law_and_de_article_share,citations_per_paper,start_pubyear,end_pubyear
kriminologisches journal,['0341-1966'],79,2442,74,0.94,30.91,2008,2013
tijdschrift voor rechtsgeschiedenis-revue d histoire du droit-the legal history review,['0040-7585'],279,6028,44,0.16,21.61,2008,2023


Cleanup: 

In [11]:
%%sql
drop table if exists {{result_schema}}.tmp_wos_law_de_items_{{version_name}};
drop table if exists {{result_schema}}.tmp_wos_law_de_journals_{{version_name}};

Running query in 'kb_prod'

++
||
++
++

## <a id="list-scopus"></a> Scopus

Specifications: 
* all journals with at least one item with: class_name = 'Law', source type = 'Journal', item type = 'Article', language = 'ger' --> {{result_schema}}.eva_scp_de_law_journals_trivial_search_{{version_name}}
* from this result only the journals with at least 10 german law articles and at least 10% of all items german law article

In [13]:
%%sql
drop table if exists {{result_schema}}.tmp_scp_law_de_items_{{version_name}};
create table {{result_schema}}.tmp_scp_law_de_items_{{version_name}} as (
select item_id, doi, pubyear, scopus_source_id, source_title, publisher_hash 
from {{scp_bdb_schema}}.items 
where 'ger' = any(languages) and 'Law' =any(class_name) and source_type='Journal' and 'Article' = any(item_type)
);
create index on {{result_schema}}.tmp_scp_law_de_items_{{version_name}}(item_id);
create index on {{result_schema}}.tmp_scp_law_de_items_{{version_name}}(publisher_hash);

drop table if exists {{result_schema}}.tmp_scp_law_de_journals_{{version_name}};
create table {{result_schema}}.tmp_scp_law_de_journals_{{version_name}} as (
select b.scopus_source_id, array_agg( distinct b.source_title) as source_title, array_agg(distinct sn_c) as issn, 
count(distinct b.item_id) as article_count, count(distinct a.item_id) as law_and_de, 
sum(cit_all_years) as cited_by_count, min(b.pubyear) as start_pubyear, max(b.pubyear) as end_pubyear
from {{result_schema}}.tmp_scp_law_de_items_{{version_name}} a 
join {{scp_bdb_schema}}.items b 
on a.source_title = b.source_title  and 'Article' = any(b.item_type) -- alle items deren source title in den law und de items auftaucht
left join {{scp_bdb_schema}}.issn_isbn c 
on c.item_id = b.item_id 
group by b.scopus_source_id);

drop table if exists {{result_schema}}.eva_scp_de_law_journals_trivial_search_{{version_name}};
create table {{result_schema}}.eva_scp_de_law_journals_trivial_search_{{version_name}} as (
select scopus_source_id, source_title, issn, 
article_count, cited_by_count, law_and_de, 
case when article_count > 0 then round(law_and_de/article_count::decimal,2) else null end as law_and_de_article_share, 
case when article_count > 0 then round(cited_by_count/article_count::decimal,2) else null end as citations_per_paper
from  {{result_schema}}.tmp_scp_law_de_journals_{{version_name}});

drop table if exists {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}};
create table {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}} as (
select * 
from {{result_schema}}.eva_scp_de_law_journals_trivial_search_{{version_name}}  
where law_and_de >= 10 and law_and_de_article_share >= 0.1 
order by citations_per_paper desc
);

drop table if exists {{result_schema}}.tmp_scp_law_de_items_{{version_name}};
drop table if exists {{result_schema}}.tmp_scp_law_de_journals_{{version_name}};

Running query in 'kb_prod'

7977 rows affected.

55 rows affected.

55 rows affected.

23 rows affected.

++
||
++
++

Number of resulting journals: 

In [14]:
%%sql
select count(*) as number_of_journals from  {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

1 rows affected.

number_of_journals
23


Details: 

In [15]:
%%sql
select * from  {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

23 rows affected.

scopus_source_id,source_title,issn,article_count,cited_by_count,law_and_de,law_and_de_article_share,citations_per_paper
21979,['Deutsche Lebensmittel-Rundschau'],['0012-0413'],1739,7372264,1478,0.85,4239.37
7200153135,"['Forensische Psychiatrie, Psychologie, Kriminologie']","['1862-7072', '1862-7080']",523,1401516,513,0.98,2679.76
20905,['Kriminalistik'],['0023-4699'],2601,3636001,2599,1.00,1397.92
14109,['Recht und Psychiatrie'],['0724-2247'],369,280876,284,0.77,761.18
16100154738,"['Zeitschrift der Savigny-Stiftung fur Rechtsgeschichte, Romanistische Abteilung']","['0323-4096', '2304-4934']",603,409500,500,0.83,679.10
21100239827,['Monatsschrift fur Kriminologie und Strafrechtsreform'],"['0026-9301', '2366-1968']",323,176080,284,0.88,545.14
18498,"['Zeitschrift der Savigny-Stiftung fur Rechtsgeschichte, Germanistische Abteilung']","['0323-4045', '2304-4861']",398,119733,373,0.94,300.84
18700156726,['Kriminologisches Journal'],['0341-1966'],207,47472,184,0.89,229.33
21100236610,['Rechtsgeschichte'],"['1619-4993', '2195-9617']",309,62640,174,0.56,202.72
5800157711,['Archiv fur Rechts- und Sozialphilosophie'],"['0001-2343', '2363-5606']",273,39909,159,0.58,146.19


## <a id="list-openalex"></a> OpenAlex

Specifications: 
* all journals with at least one item with: subfield_id = '3308', source type = 'Journal', language = 'de' --> {{result_schema}}.eva_openalex_de_law_journals_trivial_search_{{version_name}}
* from this result only the journals with at least 10 german law articles and at least 10% of all items german law article

In [23]:
%%sql
select * from {{openalex_rep_schema}}.subfields where lower(display_name) like '%law%'

Running query in 'kb_prod'

2 rows affected.

id,display_name
2308,"Management, Monitoring, Policy and Law"
3308,Law


--> take 3308 only.

In [10]:
%%sql
drop table if exists {{result_schema}}.tmp_law_{{version_name}};
create table {{result_schema}}.tmp_law_{{version_name}} as (select distinct work_id 
from {{openalex_rep_schema}}.works_topics b
join {{openalex_rep_schema}}.topics c
on b.topic_id = c.id 
where c.subfield_id = '3308');
create index on {{result_schema}}.tmp_law_{{version_name}}(work_id);


Running query in 'kb_prod'

3425319 rows affected.

++
||
++
++

In [ ]:
%%sql
alter table {{result_schema}}.tmp_law_{{version_name}} add column article_is_de boolean;

In [ ]:
%%sql
update {{result_schema}}.tmp_law_{{version_name}}  
set article_is_de = false;

update {{result_schema}}.tmp_law_{{version_name}} set article_is_de = true 
where work_id in (select id 
from {{openalex_rep_schema}}.works 
where language = 'de'); 

drop table if exists {{result_schema}}.tmp_law_II_{{version_name}}; 
create table {{result_schema}}.tmp_law_II_{{version_name}} as (
select c.id as source_id, display_name, works_count, cited_by_count, count(distinct a.work_id) as law_and_de, c.type as source_type   
from {{result_schema}}.tmp_law_{{version_name}} a
join {{openalex_rep_schema}}.works_locations b 
on a.work_id = b.work_id 
join {{openalex_rep_schema}}.sources c 
on b.source_id = c.id 
where article_is_de = true 
group by c.id, display_name, works_count, cited_by_count, c.type);

create index on {{result_schema}}.tmp_law_II_{{version_name}}(source_id);

update {{result_schema}}.tmp_law_II_{{version_name}} a 
set works_count = (select count(*)
from {{openalex_rep_schema}}.works_locations 
where source_id = a.source_id);


drop table if exists {{result_schema}}.eva_openalex_de_law_journals_trivial_search_{{version_name}};
create table {{result_schema}}.eva_openalex_de_law_journals_trivial_search_{{version_name}} as (
select source_id, display_name, 
works_count, cited_by_count, law_and_de, 
case when works_count > 0 then round(law_and_de/works_count::decimal,2) else null end as law_and_de_article_share, 
case when works_count > 0 then round(cited_by_count/works_count::decimal,2) else null end as citations_per_paper
from {{result_schema}}.tmp_law_II_{{version_name}}
where lower(source_type) = 'journal'
);

drop table if exists {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}};
create table {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}} as (
select * 
from {{result_schema}}.eva_openalex_de_law_journals_trivial_search_{{version_name}}  
where law_and_de >= 10 and law_and_de_article_share >= 0.1);

Number of resulting journals: 

In [15]:
%%sql
select count(*) from {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

1 rows affected.

count
323


In [16]:
%%sql
select count(distinct display_name) from {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}};

Running query in 'kb_prod'

1 rows affected.

count
315


There are some cases where one display name for a source appears with different source ids. 
In our set this occurs 8 times where in each case one display name appeared with two source ids. These might be data errors.  

In [18]:
%%sql
select source_id, display_name 
from project_rewi.eva_openalex_de_law_journals_trivial_search_result_{version_name} 
where display_name in 
(select display_name
from project_rewi.eva_openalex_de_law_journals_trivial_search_result_{version_name} eodljtsr 
group by display_name 
having count(*)> 1) 
order by display_name

Running query in 'kb_prod'

16 rows affected.

source_id,display_name
S4306504063,Baurechtliche Blätter
S4210189108,Baurechtliche Blätter
S4210228072,Die Verwaltung
S145628082,Die Verwaltung
S4210193381,Juristische Blätter
S4306516540,Juristische Blätter
S4210233727,KUR - Kunst und Recht
S4210182874,KUR - Kunst und Recht
S4210223432,Rechtsgeschichte - Legal History
S4306525478,Rechtsgeschichte - Legal History


Details: 

In [19]:
%%sql
select source_id as openalex_journal_id, display_name, works_count as number_of_articles, law_and_de_article_share
from project_rewi.eva_openalex_de_law_journals_trivial_search_result_{version_name} 
order by works_count desc

Running query in 'kb_prod'

323 rows affected.

openalex_journal_id,display_name,number_of_articles,law_and_de_article_share
S25774322,Juristische Rundschau,14334,0.29
S4210177729,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Germanistische Abteilung,13032,0.31
S4210169614,Monatsschrift für Deutsches Recht,11235,0.34
S4210169399,ARCHIV FÜR KATHOLISCHES KIRCHENRECHT,9096,0.17
S950333007,Monatsschrift für Kriminologie und Strafrechtsreform,6999,0.11
S98369295,Medizinrecht,6857,0.20
S157638965,JuristenZeitung,6720,0.41
S4210238088,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Romanistische Abteilung,6637,0.14
S4210187269,JurPC,6454,0.14
S4210185480,Deutsche Zeitschrift für Wirtschafts- und Insolvenzrecht,5509,0.36


# <a id="coverage"></a> Coverage of the gold standard list in different data sources

## <a id="coverage-preparations"></a>Preparations

The Gröls/Gröls article contains two lists of journals: one with general german law journals, one with field specific journals. For evaluation on coverage we used a union of these journals 
as we are interested in the coverage of 'important' german law journals, never mind if they are general ones or belong to a specific subfield. 

The tables this query are based on are created from the article mentioned above (table 1 and table 2). This step has to be done only once as it creates the table from the article 
information and stays fix.

In [131]:
%%sql
create table if not exists {{result_schema}}.jura_groels_alle_zeitschriften as (
select zeitschrift, abk, array_agg(issn) as issn_list
from (
(select distinct zeitschrift, abk, issn 
from 
(select zeitschrift, abk, unnest(STRING_TO_ARRAY(issn_manuell, ',')) as issn
from project_rewi.jura_groels_tabelle1_allg_zeitschriften jgtaz 
union 
select zeitschrift, abk, unnest(STRING_TO_ARRAY(issn_manuell, ',')) as issn 
from project_rewi.jura_groels_tabelle2_fachspez_zeitschriften jgtfz)) 
)
group by zeitschrift, abk);

Running query in 'kb_prod'

++
||
++
++

## <a id="coverage-head"></a> Coverage

The following queries append columns for each datasource and append columns indicating ids in the different sources, matching is done based on issn. We also did a manual check afterwards for finding journals that could not be mapped automatically based on issn as a postprocessing step for our first version. 
In a first step a table for recording the coverage in the given version is created as a copy from the fixed table with all journals from the article.

In [132]:
%%sql
drop table if exists {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}};
create table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} as (
select zeitschrift, abk, issn_list from {{result_schema}}.jura_groels_alle_zeitschriften);
commit;

Running query in 'kb_prod'

51 rows affected.

++
||
++
++

### <a id="coverage-wos"></a> Web of Science

In [133]:
%%sql
drop table if exists {{result_schema}}.ztab_wos;

create table {{result_schema}}.ztab_wos as (
select distinct zeitschrift, abk, issn_list, array_agg(i.source_title) as wos_source_title, array_agg(distinct issn) as issn_bdb  
from (select zeitschrift, abk, issn_list, unnest(issn_list) as issn from {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}) a 
join {{wos_bdb_schema}}.issn_isbn b 
on b.sn_c=a.issn
join {{wos_bdb_schema}}.items i 
on i.item_id=b.item_id
group by zeitschrift, abk, issn_list); 

alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add wos_source_title text[];
alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add wos_issn text[];
update {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} a 
set wos_source_title = (select wos_source_title from {{result_schema}}.ztab_wos where abk=a.abk),
wos_issn = (select issn_bdb from {{result_schema}}.ztab_wos where abk=a.abk);

drop table if exists {{result_schema}}.ztab_wos;

commit;

Running query in 'kb_prod'

51 rows affected.

++
||
++
++

### <a id="coverage-scp"></a>Scopus

In [134]:
%%sql
drop table if exists {{result_schema}}.ztab_scp;
create table {{result_schema}}.ztab_scp as (
select distinct zeitschrift, abk, issn_list, array_agg(distinct issn) as issn_bdb, array_agg(distinct i.scopus_source_id)  as scp_ids
from (select zeitschrift, abk, issn_list, unnest(issn_list) as issn from {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}) a 
join {{scopus_bdb_schema}}.issn_isbn b 
on b.sn_c=a.issn 
join {{scopus_bdb_schema}}.items i 
on i.item_id=b.item_id
group by zeitschrift, abk, issn_list); 

alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add scopus_source_id text[];
alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add scp_issn text[];
update {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} a 
set scopus_source_id = (select scp_ids from {{result_schema}}.ztab_scp where abk=a.abk),
scp_issn = (select issn_bdb from {{result_schema}}.ztab_scp where abk=a.abk);

drop table if exists {{result_schema}}.ztab_scp;

commit;

Running query in 'kb_prod'

3 rows affected.

51 rows affected.

++
||
++
++

### <a id="coverage-crossref"></a> CrossRef

Regular CrossRef Updates and Snapshots are not part of the KB infrastructure so far, therefore we query the CrossRef API for the coverage information. Match is done using ISSN as in the other cases. The date the CrossRef API was queried is set as column comment.

In [135]:
import json
import urllib
import urllib.request
import datetime

today = str(datetime.datetime.now())

def get_crossref_journal(issn):
    base_url = "https://api.crossref.org/journals/"
    url = base_url + issn

    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read().decode())
            return data['message']['ISSN']
    except urllib.error.HTTPError as e:
        # print(f"HTTP Error {e.code}: {e.reason}")
        return None
    except json.JSONDecodeError:
        # print("Error decoding JSON response")
        return None

In [136]:
%%sql
alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add column if not exists crossref_issn text[];
update {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} set crossref_issn = null;
comment on column {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}.crossref_issn IS 'Crossref queried {{today}}';
commit;

Running query in 'kb_prod'

51 rows affected.

++
||
++
++

In [137]:
from IPython import get_ipython
ipython = get_ipython()

result = %sql select unnest(issn_list) from {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}

for row in result:
    issn = row[0]
    issn_list_crossref = []
    issn_list_crossref = get_crossref_journal(issn)

    if issn_list_crossref: 
        print(f'(+) issn {issn} found in CrossRef! List of CrossRef issn: {issn_list_crossref}')
        for crossref_issn in issn_list_crossref:
            update_query = f"""
            %%sql
            UPDATE {result_schema}.jura_groels_alle_zeitschriften_{version_name}
            SET crossref_issn = array_append(COALESCE(crossref_issn, '{{}}'), '{crossref_issn}')
            WHERE (lower('{issn}') = any(issn_list) or upper('{issn}') = any(issn_list)) 
            and (crossref_issn is null or not '{crossref_issn}' = any(crossref_issn))
            """
            
            ipython.run_cell(update_query)
    else:
        print(f'(-) issn {issn} not found in CrossRef')

Running query in 'kb_prod'

89 rows affected.

(+) issn 2366-097X found in CrossRef! List of CrossRef issn: ['0002-3752', '2366-097X']


Running query in 'kb_prod'

1 rows affected.

...

Running query in 'kb_prod'

++
||
++
++

Running query in 'kb_prod'

++
||
++
++

(-) issn 0017-1956 not found in CrossRef
(-) issn 0342-6971 not found in CrossRef
(-) issn 1434-9272 not found in CrossRef
(-) issn 0941-7915 not found in CrossRef
(-) issn 0029-859X not found in CrossRef
(-) issn 1863-6470 not found in CrossRef
(-) issn 0943-7525 not found in CrossRef


### <a id="coverage-openalex"></a> OpenAlex

In [138]:
%%sql
drop table if exists {{result_schema}}.ztab_openalex;
create table {{result_schema}}.ztab_openalex as (
select distinct zeitschrift, abk, issn_list, array_agg(distinct b.issn) as issn_bdb, array_agg(b.id) as openalex_ids
from (select zeitschrift, abk, issn_list, unnest(issn_list) as issn from {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}) a 
join (
select json_array_elements_text(issn -> 'issn') as issn, id from {openalex_rep_schema}.sources 
union select issn_l as issn, id from {openalex_rep_schema}.sources
) b 
on a.issn=b.issn
group by zeitschrift, abk, issn_list); 

alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add column if not exists openalex_source_id text[];
alter table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} add column if not exists openalex_issn text[];
update {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} a 
set openalex_source_id = (select openalex_ids from {{result_schema}}.ztab_openalex where abk=a.abk),
openalex_issn = (select issn_bdb from {{result_schema}}.ztab_openalex where abk=a.abk);

drop table if exists {{result_schema}}.ztab_openalex;

commit;

Running query in 'kb_prod'

32 rows affected.

51 rows affected.

++
||
++
++

### How many and which of the journals in the gold standard are in the list gained from the trivial search?  

In [20]:
%%sql
alter table {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}}   
add column if not exists in_goldstandard boolean; 
update {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = false; 
update {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = true 
where source_id in (SELECT unnest(openalex_source_id) FROM {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}); 

Running query in 'kb_prod'

323 rows affected.

32 rows affected.

++
||
++
++

Flag gold standard journals in Scopus trivial search list: 

In [7]:
%%sql
alter table {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}}   
add column if not exists in_goldstandard boolean; 
update {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = false; 
update {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = true 
where scopus_source_id in (SELECT unnest(scopus_source_id) FROM {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}); 

Running query in 'kb_prod'

23 rows affected.

5 rows affected.

++
||
++
++

Flag gold standard journals in WoS trivial search list: 

In [9]:
%%sql
alter table {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}}   
add column if not exists in_goldstandard boolean; 
update {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = false; 
update {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}}  
set in_goldstandard = true 
where source_title in (SELECT unnest(wos_source_title) FROM {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}}); 

Running query in 'kb_prod'

2 rows affected.

++
||
++
++

Number of journals in the gold standard also found with the trivial search: 

In [21]:
%%sql
select 'OpenAlex: ' as source, count(*) as in_goldstandard_groels
from {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}} 
where in_goldstandard = true;

Running query in 'kb_prod'

1 rows affected.

source,in_goldstandard_groels
OpenAlex:,32


In [12]:
%%sql
select 'Scopus: ' as source, count(*) as in_goldstandard_groels
from {{result_schema}}.eva_scp_de_law_journals_trivial_search_result_{{version_name}} 
where in_goldstandard = true;

Running query in 'kb_prod'

1 rows affected.

source,in_goldstandard_groels
Scopus:,5


In [13]:
%%sql
select 'WoS: ' as source, count(*) as in_goldstandard_groels
from {{result_schema}}.eva_wos_de_law_journals_trivial_search_result_{{version_name}} 
where in_goldstandard = true;

Running query in 'kb_prod'

1 rows affected.

source,in_goldstandard_groels
WoS:,0


For OpenAlex: try to find the 'relevant' (=in Gröls/Gröls gold standard) journals among the 323 journals found with the trivial search

...by sorting by number of articles

In [24]:
%%sql
select source_id, display_name, rank 
from (
SELECT source_id, display_name, works_count, RANK() OVER (ORDER BY works_count DESC) as rank, in_goldstandard
FROM  {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}} 
ORDER BY rank)
where in_goldstandard is true

Running query in 'kb_prod'

32 rows affected.

source_id,display_name,rank
S25774322,Juristische Rundschau,1
S4210177729,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Germanistische Abteilung,2
S4210169614,Monatsschrift für Deutsches Recht,3
S157638965,JuristenZeitung,7
S4210238088,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Romanistische Abteilung,8
S4210167899,Die Aktiengesellschaft,12
S4210191334,Zeitschrift für die gesamte Strafrechtswissenschaft,13
S4210240061,JURA - Juristische Ausbildung,18
S4210190048,Computer und Recht,21
S4210207308,KJ / Kritische Justiz,26


...by sorting by number of citations

In [25]:
%%sql
select source_id, display_name, rank 
from (
SELECT source_id, display_name, cited_by_count, RANK() OVER (ORDER BY cited_by_count DESC) as rank, in_goldstandard
FROM   {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}} 
ORDER BY rank)
where in_goldstandard is true

Running query in 'kb_prod'

32 rows affected.

source_id,display_name,rank
S25774322,Juristische Rundschau,2
S4210177729,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Germanistische Abteilung,3
S4210238088,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Romanistische Abteilung,4
S4210207308,KJ / Kritische Justiz,6
S157638965,JuristenZeitung,8
S4210191334,Zeitschrift für die gesamte Strafrechtswissenschaft,11
S108365452,Rabels Zeitschrift für ausländisches und internationales Privatrecht,15
S204358660,Der Staat,19
S4210199208,Zeitschrift für Unternehmens- und Gesellschaftsrecht,21
S4210204607,Europarecht,25


...by sorting by citations per paper

In [26]:
%%sql
select source_id, display_name, rank 
from (
SELECT source_id, display_name, citations_per_paper, RANK() OVER (ORDER BY citations_per_paper DESC) as rank, in_goldstandard
FROM   {{result_schema}}.eva_openalex_de_law_journals_trivial_search_result_{{version_name}} 
ORDER BY rank)
where in_goldstandard is true

Running query in 'kb_prod'

32 rows affected.

source_id,display_name,rank
S204358660,Der Staat,15
S4210231065,Archiv des Völkerrechts,19
S4210207308,KJ / Kritische Justiz,22
S108365452,Rabels Zeitschrift für ausländisches und internationales Privatrecht,25
S78787406,Archiv des öffentlichen Rechts,30
S4210204607,Europarecht,31
S4210238088,Zeitschrift der Savigny-Stiftung für Rechtsgeschichte Romanistische Abteilung,34
S4306504202,Betriebs-Berater,36
S4210234159,Kritische Vierteljahresschrift für Gesetzgebung und Rechtswissenschaft,38
S82779185,Archiv für die civilistische Praxis,39


## <a id="coverage-results"></a> Coverage results

### <a id="coverage-results-one"> Checkout results for one version

The comparison of different versions required a table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} for each version. 
The parameters specify the versions to compare (defined by version name).

In [140]:
version_name = "202408"
version_schema = "project_rewi"

In [141]:
#### Overview

In [142]:
%%sql
select * from (
select '{{version_name}}' as version, 'wos' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}})::decimal,2) as coverage_percent from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}} where wos_issn is not null
    union
select '{{version_name}}' as version, 'scopus' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}})::decimal,2) as coverage_percent from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}} where scp_issn is not null
    union
select '{{version_name}}' as version, 'CrossRef' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}})::decimal,2) as coverage_percent from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}} where crossref_issn is not null
    union
select '{{version_name}}' as version, 'openalex' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}})::decimal,2) as coverage_percent from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}} where openalex_issn is not null
) 
order by data_source, version

Running query in 'kb_prod'

4 rows affected.

version,data_source,coverage,coverage_percent
202408,CrossRef,24,47.06
202408,openalex,32,62.75
202408,scopus,3,5.88
202408,wos,0,0.00


#### Details

In [144]:
%%sql
select * from {{version_schema}}.jura_groels_alle_zeitschriften_{{version_name}}

Running query in 'kb_prod'

51 rows affected.

zeitschrift,abk,issn_list,wos_source_title,wos_issn,scopus_source_id,scp_issn,crossref_issn,openalex_source_id,openalex_issn
Betriebs-Berater,BB,"['0947-0581', '0340-7918']",None,None,None,None,None,['S4306504202'],['0340-7918']
Zeitschrift für Wirtschaftsrecht,ZIP,['0723-9416'],None,None,None,None,None,None,None
Europäische Grundrechtezeitschrift,EuGRZ,"['2750-2767', '0341-9800']",None,None,None,None,None,None,None
Arbeit und Recht,AuR,"['2942-0598', '0003-7648']",None,None,None,None,None,None,None
Zeitschrift für das gesamte Handels- und Wirtschaftsrecht,ZHR,['0044-2437'],None,None,None,None,None,None,None
Zeitschrift für das gesamte Familienrecht,FamRZ,['0044-2410'],None,None,None,None,None,None,None
Neue Zeitschrift für Strafrecht,NStZ,['0720-1753'],None,None,None,None,None,['S4306520745'],['0720-1753']
Zeitschrift für Wirtschafts- und Steuerstrafrecht,wistra,['0721-6890'],None,None,None,None,None,None,None
Deutsches Steuerrecht,DStR,"['0012-1347', '0949-7676']",None,None,None,None,None,None,None
Juristische Arbeitsblätter,JA,['0022-6904'],None,None,None,None,None,None,None


### <a id="coverage-results-two"> Compare different versions

The comparison of different versions required a table {{result_schema}}.jura_groels_alle_zeitschriften_{{version_name}} for each version. 
The parameters specify the versions to compare (defined by version name).

In [145]:
version_name_1 = "202404"
version_schema_1 = "project_rewi"
version_name_2 = "202408"
version_schema_2 = "project_rewi"

In [146]:
%%sql
select * from (
select '{{version_name_1}}' as version, 'wos' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}})::decimal,2) as coverage_percent from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}} where wos_issn is not null
union
select '{{version_name_2}}' as version, 'wos' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}})::decimal,2) as coverage_percent from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}} where wos_issn is not null

    union
select '{{version_name_1}}' as version, 'scopus' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}})::decimal,2) as coverage_percent from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}} where scp_issn is not null
union
select '{{version_name_2}}' as version, 'scopus' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}})::decimal,2) as coverage_percent from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}} where scp_issn is not null

    union
select '{{version_name_1}}' as version, 'CrossRef' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}})::decimal,2) as coverage_percent from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}} where crossref_issn is not null
union
select '{{version_name_2}}' as version, 'CrossRef' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}})::decimal,2) as coverage_percent from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}} where crossref_issn is not null

    union
select '{{version_name_1}}' as version, 'openalex' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}})::decimal,2) as coverage_percent from {{version_schema_1}}.jura_groels_alle_zeitschriften_{{version_name_1}} where openalex_issn is not null
union
select '{{version_name_2}}' as version, 'openalex' as data_source, count(*) as coverage, round(100*count(*)/(select count(*) from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}})::decimal,2) as coverage_percent from {{version_schema_2}}.jura_groels_alle_zeitschriften_{{version_name_2}} where openalex_issn is not null
    ) 
order by data_source, version

Running query in 'kb_prod'

8 rows affected.

version,data_source,coverage,coverage_percent
202404,CrossRef,24,47.06
202408,CrossRef,24,47.06
202404,openalex,32,62.75
202408,openalex,32,62.75
202404,scopus,3,5.88
202408,scopus,3,5.88
202404,wos,0,0.00
202408,wos,0,0.00
